In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
train = pd.read_csv("../input/ventilator-pressure-prediction/train.csv")
test = pd.read_csv("../input/ventilator-pressure-prediction/test.csv")

In [ ]:
train.head()

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.regression import setup, compare_models, blend_models, finalize_model, predict_model

In [ ]:
# def pycaret_model(train, target, test, n_select, fold, opt):
#   print('Setup Your Data....')
#   setup(data=train,
#               target=target,
#               numeric_imputation = 'mean',
#               normalize = True,
#               silent= True)
  
#   print('Comparing Models....')

#   best = compare_models(sort=opt, n_select=n_select, fold = fold)
#   print('Blending Models....')
#   blended = blend_models(estimator_list= best, fold=fold, optimize=opt)
#   pred_holdout = predict_model(blended)
#   print('Finallizing Models....')
#   final_model = finalize_model(blended)
#   print('Done...!!!')
#   pred_esb = predict_model(final_model, test)
#   re = pred_esb['Label']

#   return re

In [ ]:
n_select = 2
fold = 5 
opt = "MAE"

In [ ]:
print('Setup Your Data....')
# setup(data=train,target='pressure',numeric_imputation = 'mean',normalize = True,silent= True)
setup(data=train,target='pressure',normalize = True,normalize_method = 'robust',silent= True,use_gpu=True,
      session_id = 42,numeric_imputation = 'mean')

In [ ]:
# print('Comparing Models....')
# best = compare_models(sort=opt, n_select=n_select, fold = fold)

In [ ]:
from pycaret.regression import create_model

In [ ]:
model_list = []
lgbm = create_model(estimator = 'lightgbm', fold = fold)
xgb = create_model(estimator = 'xgboost', fold = fold)
cat = create_model(estimator = 'catboost', fold = fold)

model_list.append(lgbm)
model_list.append(xgb)
model_list.append(cat)

print('Blending Models....')
blended = blend_models(estimator_list= model_list, fold=fold,optimize=opt)

In [ ]:
# print('Blending Models....')
# blended = blend_models(estimator_list= best, fold=fold, optimize=opt)

In [ ]:
pred_holdout = predict_model(blended)
print('Done...!!!')

In [ ]:
print('Finallizing Models....')
final_model = finalize_model(blended)

In [ ]:
pred_esb = predict_model(final_model, test)
result = pred_esb['Label']

In [ ]:
# result = pycaret_model(train, 'pressure', test, 4, 3, 'MAE')

In [ ]:
sub = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")
sub["id"] = test["id"]
sub["pressure"] = result


In [ ]:
sub.to_csv('sub.csv',index=False)